In [1]:
#!/usr/bin/evn python

"""
CMSC733 Spring 2019: Classical and Deep Learning Approaches for
Geometric Computer Vision
Project1: MyAutoPano: Phase 1 Starter Code

Author(s): 
Chahat Deep Singh (chahat@terpmail.umd.edu) 
PhD Student in Computer Science,
University of Maryland, College Park

Nitin J. Sanket (nitinsan@terpmail.umd.edu)
PhD Candidate in Computer Science,
University of Maryland, College Park
"""

#Code starts here:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.feature import peak_local_max

In [2]:
def readImageSet(folder_name):
    print("Reading images from ", folder_name)
    images = []
    files = os.listdir(folder_name)	
    files = sorted(files)
    print("Found ", files)
    for file in files:
        image_path = folder_name + "/" + file
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)			
        else:
            print("Error in loading image ", image)

    return images


In [3]:
def displayImages(image_array, name):

    image_array = makeImageSizeSame(image_array)
    concat = image_array[0].copy()

    for l in range(1,len(image_array)):
        image = image_array[l]
        concat = np.concatenate((concat,image), axis = 1)
        
    cv2.imshow(name, concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [4]:
def detectCorners(images, choice):
    print("detecting corners ...")
    detected_corners = []
    cmaps = []
    corner_images = []
    for i in images:
        image = i.copy()
        gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        gray_image = np.float32(gray_image)


        if(choice == 1):
            print("using Harris corner detection method.")
            corner_strength = cv2.cornerHarris(gray_image,2,3,0.04)
            corner_strength[corner_strength<0.01*corner_strength.max()] = 0
            detected_corner = np.where(corner_strength>0.001*corner_strength.max())
            detected_corners.append(detected_corner)
            cmaps.append(corner_strength)
            image[corner_strength > 0.001*corner_strength.max()]=[0,0,255]
            corner_images.append(image)
        else:
            print("using Shi-Tomashi corner detection method.")
            dst = cv2.goodFeaturesToTrack(gray_image, 1000 ,0.01, 10)
            dst = np.int0(dst)
            detected_corners.append(dst)
            for c in dst:
                x,y = c.ravel()
                cv2.circle(image,(x,y),3,(0, 0, 255),-1) 
                          
            corner_images.append(image)
            cmap = np.zeros(gray_image.shape) #not sure what to do
            cmaps.append(cmap)
    #filter detected corners
    #remove the corner one
    return detected_corners, cmaps, corner_images

In [5]:
def getFeatureDescriptor(gray_image,x,y, patch_size=40):
    patch = gray_image[x-patch_size//2:x+patch_size//2, y-patch_size//2:y+patch_size//2] 
    # gaussian blur
    patch = cv2.GaussianBlur(patch,(3,3),0)
    # subsample to 20% size or 1/5th
    patch = cv2.resize(patch, None, fx=0.2, fy=0.2, interpolation = cv2.INTER_CUBIC)
    feature = patch.reshape(-1)
    feature = (feature-feature.mean())/ np.std(feature)
    return feature

In [6]:
def getPairs(image_1, image_2, all_corners_1, all_corners_2, patch_size = 40, alpha = 0.8):

    gray_image1 = cv2.cvtColor(image_1,cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(image_2,cv2.COLOR_BGR2GRAY)

    width1, height1 = image_1.shape[:2]
    width2, height2 = image_2.shape[:2]

    print("width = ", width1, ", height = ", height1)
    print("width = ", width2, ", height = ", height2)

    features_1,features_2 = [], []
    corners_1, corners_2 = [],[]

    print("all corner 1 len = ", len(all_corners_1))
    print("all corner 2 len = ", len(all_corners_2))
    
    for corner in all_corners_1:
        x,y = corner.ravel()
        
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height1) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width1):
            features_1.append(getFeatureDescriptor(gray_image1, y,x)) 
            corners_1.append([x,y])
        else:
            #print("ignored x, y", x, y)
            pass

    for corner in all_corners_2:
        x,y = corner.ravel()
        if (x - (patch_size / 2) > 0) & (x + (patch_size / 2) < height2) & (y - (patch_size / 2) > 0) & (y + (patch_size / 2) < width2):
            features_2.append(getFeatureDescriptor(gray_image2, y,x)) 
            corners_2.append([x,y]) 

    matched_pairs, match_level = [], []
    for i, feat_1 in enumerate(features_1):
        ssd = []  
        for j, feat_2 in enumerate(features_2):
            ssd.append(np.sum((feat_1 - feat_2)**2))
        top_matche = np.argmin(ssd)
        #if ssd[top_matches[0]] / ssd[top_matches[1]] < alpha:   
            #matched_pairs.append([corners_1[i] , corners_2[top_matches[0]]])
        matched_pairs.append([corners_1[i] , corners_2[top_matche]]) 
    print("matched pairs num = ", len(matched_pairs))
    matched_pairs = np.array(matched_pairs)
    return matched_pairs

In [7]:
def makeImageSizeSame(images):
    sizes = []
    for image in images:
        x, y, ch = image.shape
        sizes.append([x, y, ch])

    sizes = np.array(sizes)
    x_target, y_target, _ = np.max(sizes, axis = 0)
    
    images_resized = []

    for i, image in enumerate(images):
        image_resized = np.zeros((x_target, y_target, sizes[i, 2]), np.uint8)
        image_resized[0:sizes[i, 0], 0:sizes[i, 1], 0:sizes[i, 2]] = image
        images_resized.append(image_resized)

    return images_resized

In [8]:
def showMatches(image_1, image_2, matched_pairs, partition_width = 20):

    image_1, image_2 = makeImageSizeSame([image_1, image_2])

    concat = np.concatenate((image_1, image_2), axis = 1)
    corners_1 = matched_pairs[:,0].copy()
    corners_2  = matched_pairs[:,1].copy()
    corners_2[:,0] += image_1.shape[1]

    for (x1,y1) , (x2,y2) in zip(corners_1, corners_2):
        cv2.line(concat, (x1,y1), (x2,y2), (0, 0, 255), 1)
    
      
    cv2.imshow('Feature_matches', concat)
    cv2.waitKey() 
    cv2.destroyAllWindows()


In [9]:
def testShowMatches(image_1, image_2, partition_width = 20):
    matched_pairs= []
    I = np.linspace(10, 100, 10)
    for i in I:
        x1 = i
        y1 = i
        corner1 = np.int0(np.array([x1, y1]))

        x2 = i
        y2 = i
        corner2 = np.int0(np.array([x2, y2]))
        matched_pairs.append([corner1 , corner2])

    matched_pairs = np.array(matched_pairs)
    showMatches(image_1, image_2, matched_pairs, partition_width = 20)


In [10]:
def filterOutliers(matched_pairs, outliers, accuracy, thresh):

    set1 = matched_pairs[:, 0]
    set2 = matched_pairs[:, 1]

    N_best = 0
    H_best = np.zeros([3, 3])
    
    e = outliers / set1.shape[0]
    s = 4
    p = accuracy
    iterations = np.log(1 - p) / np.log(1 - np.power((1 - e), s))
    iterations = np.int(iterations)
    iterations = 4000

    filtered_pair_indices = []

    print("iterations = ", iterations)
    for i in range(iterations):
        #randomly select four points
        n_rows = set1.shape[0]
        random_indices = np.random.choice(n_rows, size=4)

        set1_random = set1[random_indices]
        set2_random = set2[random_indices]
              
        #compute homography
        H = cv2.getPerspectiveTransform(np.float32(set1_random), np.float32(set2_random))

        set1_dash = np.vstack((set1[:,0], set1[:,1], np.ones([1, n_rows])))
        set1_transformed_dash = np.dot(H, set1_dash)
        
        t1 = set1_transformed_dash[0,:]/set1_transformed_dash[2,:]
        t2 = set1_transformed_dash[1,:]/set1_transformed_dash[2,:]

        set1_transformed = np.array([t1, t2]).T
        #print(set1_transformed.shape)

        E = calculateError(set2, set1_transformed)
        
     
        E[E <= thresh] = 1
        E[E > thresh] = 0
    
    
        N = np.sum(E)


        if N > N_best:
            N_best = N
            H_best = H
            filtered_pair_indices = np.where(E == 1)
    
    filtered_set1 =  set1[filtered_pair_indices]
    filtered_set2 =  set2[filtered_pair_indices]

    print("Number of pairs after filtering = ", filtered_set1.shape[0])

    filter_matched_pairs = np.zeros([filtered_set1.shape[0], filtered_set1.shape[1], 2])

    filter_matched_pairs[:, 0, :] = filtered_set1
    filter_matched_pairs[:, 1, :] = filtered_set2

    filter_matched_pairs = filter_matched_pairs.astype(int)

    H_inbuit, _ = cv2.findHomography(set1, set2)
    print("inbuilt = ", H_inbuit)
    print("Computed = ", H_best)

    return H_best, filter_matched_pairs

    

In [11]:
def calculateError(set1, set2):
   
    E = np.zeros(set1.shape[0])
    tmp = set2 - set1
    num = set1.shape[0]

    for n in range(num):
        E[n] = np.linalg.norm(tmp[n])
    return E


In [12]:
def AdaptiveNonMaximalSuppression(images, C_maps, N_best):

    anms_img = []
    anms_corners = []
    for i,image in enumerate(images):

        cmap = C_maps[i]
        local_maximas = peak_local_max(cmap, min_distance=10)
        n_strong = local_maximas.shape[0]
        
        r = [np.Infinity for i in range(n_strong)]
        x=np.zeros((n_strong,1))
        y=np.zeros((n_strong,1))
        eu_dist = 0

        for i in range(n_strong):
            for j in range(n_strong):
                x_j = local_maximas[j][0]
                y_j = local_maximas[j][1]

                x_i = local_maximas[i][0]
                y_i = local_maximas[i][1]

                if(cmap[x_j, y_j] > cmap[x_i, y_i]):
                    eu_dist = np.square(x_j - x_i) + np.square(y_j - y_i)
                if r[i] > eu_dist:
                    r[i] = eu_dist
                    x[i] = x_j
                    y[i] = y_j

        index = np.argsort(r)
        index = np.flip(index)
        index = index[0:N_best]
        x_best=np.zeros((N_best,1))
        y_best=np.zeros((N_best,1))

        for i in range(N_best):
            x_best[i] = np.int0(x[index[i]])
            y_best[i] = np.int0(y[index[i]]) 
            cv2.circle(image, (y_best[i], x_best[i]), 3, (0, 255, 0), -1)

        anms_corner = np.int0(np.concatenate((x_best, y_best), axis = 1))
        anms_corners.append(anms_corner)
        anms_img.append(image)
    return anms_corners, anms_img

In [13]:
def stitchImagePairs(image0, image1, H):

    #stitch image 0 on image 1
    print("shapes")
    print(image0.shape)
    print(image1.shape)
    

    h0 ,w0 ,_ = image0.shape
    h1 ,w1 ,_ = image1.shape

    points_on_image0 = np.float32([[0, 0], [0, h0], [w0, h0], [w0, 0]]).reshape(-1,1,2)
    points_on_image0_transformed = cv2.perspectiveTransform(points_on_image0, H)
    print("transformed points = ", points_on_image0_transformed)
    points_on_image1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1,1,2)

    points_on_merged_images = np.concatenate((points_on_image0_transformed, points_on_image1), axis = 0)
    points_on_merged_images_ = []

    for p in range(len(points_on_merged_images)):
        points_on_merged_images_.append(points_on_merged_images[p].ravel())

    points_on_merged_images_ = np.array(points_on_merged_images_)

    x_min, y_min = np.int0(np.min(points_on_merged_images_, axis = 0))
    x_max, y_max = np.int0(np.max(points_on_merged_images_, axis = 0))

    print("min, max")
    print(x_min, y_min)
    print(x_max, y_max)

    # overlap_area = cv2.polylines(image1,[np.int32(points_on_image0_transformed)],True,255,3, cv2.LINE_AA) 
    # cv2.imshow("original_image_overlapping.jpg", overlap_area)
    # cv2.waitKey() 
    # cv2.destroyAllWindows()
    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    image0_transformed = cv2.warpPerspective(image0, np.dot(H_translate, H), (x_max-x_min, y_max-y_min))
    image0_transformed[-y_min:-y_min+h1, -x_min: -x_min+w1] = image1
    return(image0_transformed)

In [14]:
def stitchImageArray(image_array, H_array):
    
    N = len(image_array)

    target_image = image_array[N-1]

    ht, wt, _ = target_image.shape
    points_on_target_image = np.float32([[0, 0], [0, ht], [wt, ht], [wt, 0]]).reshape(-1,1,2)

    points = points_on_target_image
    
    images_transformed = []
    H_translate_array = []

    for n in range(N-1):
        image = image_array[n]
        h, w, _ = image.shape
        points_on_image = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1,1,2)

        points_on_image_transformed = points_on_image 
        for m in range(n, N-1):
            H = H_array[m]
            points_on_image_transformed = cv2.perspectiveTransform(points_on_image_transformed, H)
        
        points = np.concatenate((points, points_on_image_transformed), axis = 0)

    points_ = []
    for p in range(len(points)):
        points_.append(points[p].ravel())

    points_ = np.array(points_)

    x_min, y_min = np.int0(np.min(points_, axis = 0))
    x_max, y_max = np.int0(np.max(points_, axis = 0))

    H_translate = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]) # translate

    for n in range(N-1):
        image = image_array[n]
        image_transformed = image
        for m in range(n, N-1):
            H = H_array[m]
            image_transformed = cv2.warpPerspective(image_transformed, np.dot(H_translate,H), (x_max-x_min, y_max-y_min))

        #image_transformed = cv2.warpAffine(image_transformed, H_translate, (x_max-x_min, y_max-y_min))
        images_transformed.append(image_transformed)

    final_output = np.zeros((y_max-y_min, x_max-x_min, 3), np.uint8)
    for i in range(len(images_transformed)):
        final_output = final_output + images_transformed[i]

    return images_transformed

In [15]:
def main():
    # Add any Command Line arguments here
    # Parser = argparse.ArgumentParser()
    # Parser.add_argument('--NumFeatures', default=100, help='Number of best features to extract from each image, Default:100')
    
    # Args = Parser.parse_args()
    # NumFeatures = Args.NumFeatures
    

    """
    Read a set of images for Panorama stitching
    """
    print("main")
    folder_name = "/home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1"
    use_harris = False
    images = readImageSet(folder_name)
    displayImages(images, "image_set")
    N_images = len(images)

    choice = 2
    if use_harris:
        choice = 1

    image0 = images[0]

    for i in range(1, N_images):

        print("processing image ", i)
        image1 = images[i] 

        image_pair = [image0, image1]
        
        detected_corners, cmaps, corner_images = detectCorners(image_pair, choice)
        displayImages(corner_images, "corners")
        """
        Perform ANMS: Adaptive Non-Maximal Suppression
        Save ANMS output as anms.png
        """
        if (choice == 1):
            print("Applying ALMS.")
            detected_corners, anms_image = AdaptiveNonMaximalSuppression(image_pair, cmaps, 500)
            displayImages(anms_image, "anms_output")
        else:
            print("goodFeaturesToTrack is already using ALMS.") #review

        detected_corners0 = detected_corners[0]
        detected_corners1 = detected_corners[1]
            
        matched_pairs = getPairs(image0, image1, detected_corners0, detected_corners1, patch_size = 40, alpha = 0.9 )
        showMatches(image0, image1, matched_pairs, partition_width = 20)
        """
        Refine: RANSAC, Estimate Homography
        """
        H,filtered_matched_pairs = filterOutliers(matched_pairs, 20, 0.9, 5)
        showMatches(image0, image1, filtered_matched_pairs, partition_width = 20)
        """
        Image Warping + Blending
        Save Panorama output as mypano.png
        """
        stitched_image = stitchImagePairs(image0, image1, H)
        cv2.imshow("pano.jpg", stitched_image)
        cv2.waitKey() 
        cv2.destroyAllWindows()
        cv2.imwrite("Results/pano.png", stitched_image)
        image0 = stitched_image
        


In [17]:
if __name__ == '__main__':
    main()

main
Reading images from  /home/sakshi/courses/CMSC733/sakshi_p1/Phase1/Data/Train/Set1
Found  ['1.jpg', '2.jpg', '3.jpg']
processing image  1
detecting corners ...
using Shi-Tomashi corner detection method.
using Shi-Tomashi corner detection method.
goodFeaturesToTrack is already using ALMS.
width =  450 , height =  600
width =  450 , height =  600
all corner 1 len =  616
all corner 2 len =  711
matched pairs num =  517
iterations =  4000
Number of pairs after filtering =  212
inbuilt =  [[-6.51661112e-01 -2.28634018e+00  6.49829969e+02]
 [-1.86012425e-01 -6.53463614e-01  1.85314843e+02]
 [-9.86728387e-04 -3.54897705e-03  1.00000000e+00]]
Computed =  [[ 1.14117538e+00  1.00295222e-01 -4.45012977e+01]
 [ 1.56297848e-02  1.15818181e+00 -2.68742151e+02]
 [ 1.95599725e-05  3.87205958e-04  1.00000000e+00]]
shapes
(450, 600, 3)
(450, 600, 3)
transformed points =  [[[-4.4501297e+01 -2.6874216e+02]]

 [[ 5.3783804e-01  2.1498083e+02]]

 [[ 5.7786603e+02  2.2076074e+02]]

 [[ 6.3277765e+02 -2.